### df_loc (Canada)

We will use census Distribution Areas as proxies for neighborhoods for cities in Canada.  In previous work where the Forward Sortation Areas (first three characters of the postal code) were used as neighborhood proxies, the sizes of many areas were quite large (several kilometers across) and therefore are likely internally non-homogeneous from a features perspective at the walking-distance (500 m) length scale.  To convert to neighborhood names we can look up the associated census tract as seen on [this](https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods) Wikipedia page.

File lda_000b16g_e.gml was downloaded from the [Statistics Canada: Boundary Files](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm) site.

Exploring the gml file and computing the area and centroid of the distribution areas can be done using the [osgeo library](https://pcjericks.github.io/py-gdalogr-cookbook/geometry.html#quarter-polygon-and-create-centroids).

File T1901EN.CSV was downloaded from the [Canadian Census Data](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm) site

Exploring the gml file and computing the area and centroid of the distribution areas can be done using the [osgeo library](https://pcjericks.github.io/py-gdalogr-cookbook/geometry.html#quarter-polygon-and-create-centroids).

    '''python

    # Exporting to geojson
    from osgeo import ogr

    # Create test polygon
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(1179091.1646903288, 712782.8838459781)
    ring.AddPoint(1161053.0218226474, 667456.2684348812)
    ring.AddPoint(1214704.933941905, 641092.8288590391)
    ring.AddPoint(1228580.428455506, 682719.3123998424)
    ring.AddPoint(1218405.0658121984, 721108.1805541387)
    ring.AddPoint(1179091.1646903288, 712782.8838459781)
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)

    geojson = poly.ExportToJson()
    print geojson
    
    
    '''python
    # Get centroid
    import ogr

        # Given a test polygon
    poly_Wkt= "POLYGON((-107.42631019589980212 40.11971708125970082,-107.42455436683293613 40.12061219666851741,-107.42020981542387403 40.12004414402532859,-107.41789122063043749 40.12149008687303819,-107.41419947746419439 40.11811617239460048,-107.41915181585792993 40.11761695654455906,-107.41998470913324581 40.11894245264452508,-107.42203317637793702 40.1184088144647788,-107.42430674991324224 40.1174448122981957,-107.42430674991324224 40.1174448122981957,-107.42631019589980212 40.11971708125970082))"
    geom_poly = ogr.CreateGeometryFromWkt(poly_Wkt)


    geom_poly_envelope = geom_poly.GetEnvelope()
    minX = geom_poly_envelope[0]
    minY = geom_poly_envelope[2]
    maxX = geom_poly_envelope[1]
    maxY = geom_poly_envelope[3]

    '''
    coord0----coord1----coord2
    |           |           |
    coord3----coord4----coord5
    |           |           |
    coord6----coord7----coord8
    '''
    coord0 = minX, maxY
    coord1 = minX+(maxX-minX)/2, maxY
    coord2 = maxX, maxY
    coord3 = minX, minY+(maxY-minY)/2
    coord4 = minX+(maxX-minX)/2, minY+(maxY-minY)/2
    coord5 = maxX, minY+(maxY-minY)/2
    coord6 = minX, minY
    coord7 = minX+(maxX-minX)/2, minY
    coord8 = maxX, minY

    ringTopLeft = ogr.Geometry(ogr.wkbLinearRing)
    ringTopLeft.AddPoint_2D(*coord0)
    ringTopLeft.AddPoint_2D(*coord1)
    ringTopLeft.AddPoint_2D(*coord4)
    ringTopLeft.AddPoint_2D(*coord3)
    ringTopLeft.AddPoint_2D(*coord0)
    polyTopLeft = ogr.Geometry(ogr.wkbPolygon)
    polyTopLeft.AddGeometry(ringTopLeft)


    quaterPolyTopLeft = polyTopLeft.Intersection(geom_poly)
    centroidTopLeft = quaterPolyTopLeft.Centroid()
    '''
    
    '''python
    # Create geometry from gml
    from osgeo import ogr

    gml = """<gml:Point xmlns:gml="http://www.opengis.net/gml"><gml:coordinates>108420.33,753808.59</gml:coordinates></gml:Point>"""
    point = ogr.CreateGeometryFromGML(gml)
    print "%d,%d" % (point.GetX(), point.GetY())
    '''
Note: must install gdal module

In [31]:
%%time

from osgeo import ogr

with open('lda_000b16g_e.gml','r') as f:
    geo_gml = f.read()

Wall time: 1.82 s


In [35]:
len(geo_gml)

465102295

In [39]:
g = ogr.CreateGeometryFromGML(geo_gml)
print(g)

None


In [32]:
from osgeo import ogr

source = ogr.Open('lda_000b16g_e.gml')

In [85]:
type(source)

osgeo.ogr.DataSource

In [36]:
dir(source)
s2=source.GetLayer(0)
s2

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x0000023F3C018360> >

In [50]:
len(s2)

56589

In [56]:
s4 = s2[0]
s4

<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x0000023F3BB6F6C0> >

In [60]:
s4.GetGeometryRef()

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x0000023F3C018BA0> >

In [89]:
print(s4.GetGeometryRef())

POLYGON ((8976851.14856922 2149576.54285611,8976818.14856922 2149509.53714611,8976752.55428422 2149542.32857111,8976560.31999922 2149661.59143111,8976490.55428422 2149705.07428611,8976354.78571422 2149785.82285611,8976272.75142922 2149821.67714611,8976190.13714422 2149866.79143111,8976195.66571422 2149883.50571611,8976260.85714422 2150016.91714611,8976273.13142922 2150051.04285611,8976272.92285422 2150083.44285611,8976265.71428422 2150121.91143111,8976382.41428422 2150157.02571611,8976439.10285422 2150180.22000111,8976482.51714422 2150206.11714611,8976587.63714422 2150288.98000111,8976596.70571422 2150291.94857111,8976668.80571422 2150336.58000111,8976722.28571422 2150362.41714611,8976780.99428422 2150389.63143111,8976892.28285422 2150426.98857111,8977012.79714422 2150458.65714611,8977072.04571422 2150475.00571611,8977101.92571422 2150483.54571611,8977135.12571422 2150492.93428611,8977335.04285422 2150545.85714611,8977346.74571422 2150484.42285611,8977358.09428422 2150430.22571611,8977

In [99]:
print(s2.GetSpatialRef())
print(s4.GetGeometryRef().GetSpatialReference())

PROJCS["NAD83 / Statistics Canada Lambert",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["latitude_of_origin",63.390675],
    PARAMETER["central_meridian",-91.8666666666667],
    PARAMETER["standard_parallel_1",49],
    PARAMETER["standard_parallel_2",77],
    PARAMETER["false_easting",6200000],
    PARAMETER["false_northing",3000000],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","3347"]]
PROJCS["NAD83 / Statistics Canada Lambert",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPH

In [107]:
from osgeo import ogr
from osgeo import osr

inref = s4.GetGeometryRef().GetSpatialReference()

outref = osr.SpatialReference()
outref.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(inref, outref)

point = s4.GetGeometryRef().Centroid() # ogr.CreateGeometryFromWkt("POINT (1120351.57 741921.42)")
point.Transform(transform)

print(point.ExportToWkt())
print(point)
print(point.GetX(), point.GetY())

POINT (47.561837173953 -52.7652224969758)
POINT (47.561837173953 -52.7652224969758)
47.561837173952966 -52.765222496975774


In [61]:
s4.GetGeometryRef().Centroid()

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x0000023F3C0181B0> >

In [91]:
print(s4.GetGeometryRef().Centroid())

POINT (8976786.90073827 2150052.16961121)


In [94]:
s4.GetGeometryRef().Centroid().ExportToJson()

'{ "type": "Point", "coordinates": [ 8976786.900738267228007, 2150052.169611209537834 ] }'

In [62]:
s4.GetGeometryRef().Centroid().ExportToWkt()

'POINT (8976786.90073827 2150052.16961121)'

In [88]:
s4.GetGeometryRef().Area()

610130.9356462955

In [63]:
s2.GetLayerDefn()

<osgeo.ogr.FeatureDefn; proxy of <Swig Object of type 'OGRFeatureDefnShadow *' at 0x0000023F3C018450> >

In [127]:
layerDefinition = s2.GetLayerDefn()

for i in range(layerDefinition.GetFieldCount()):
    print(layerDefinition.GetFieldDefn(i).GetName(),f"  \t",s2.GetFeature(0).GetFieldAsString(s2.GetFeature(0).GetFieldIndex(layerDefinition.GetFieldDefn(i).GetName())))

gml_id   	 idb20585c1-6855-4944-82d0-bd4a9e0ad78e
DAUID   	 10010244
PRUID   	 10
PRNAME   	 Newfoundland and Labrador / Terre-Neuve-et-Labrador
CDUID   	 1001
CDNAME   	 Division No.  1
CDTYPE   	 CDR
CCSUID   	 1001519
CCSNAME   	 St. John's
CSDUID   	 1001519
CSDNAME   	 St. John's
CSDTYPE   	 CY
ERUID   	 1010
ERNAME   	 Avalon Peninsula
SACCODE   	 1
SACTYPE   	 1
CMAUID   	 1
CMAPUID   	 10001
CMANAME   	 St. John's
CMATYPE   	 B
CTUID   	 10004
CTNAME   	 4
ADAUID   	 10010014


See the feature name definitions [here](https://www150.statcan.gc.ca/n1/pub/92-160-g/2016002/tbl/tbl_4.13-eng.htm).
But this is for 2011.

The meaning of prefixes for NAME (N), UID (U), PUID (PU), TYPE (T), and CODE (C):
* DA U Dissemination Area unique identifier (composed of the 2-digit province/territory unique identifier followed by the 2-digit census division code and the 4-digit dissemination area code)
* PR U,N Province or territory
* CD U,N,T Census Division
* CCS U,N Census Consolidated Subdivision
* CSD U,N,T Census Subdivision
* ER U,N Economic Region
* SAC T,C Statistical Area Classification: Part of are a component of a census metropolitan area, a census agglomeration, a census metropolitan influenced zone or the territories?
* CMA U,PU,N,T Census Metropolitan Area or Census Agglomeration name, PU Uniquely identifies the provincial or territorial part of a census metropolitan area or census agglomeration (composed of the 2-digit province or territory unique identifier followed by the 3-digit census metropolitan area or census agglomeration unique identifier)
* CT U,N Census Tract within census metropolitan area/census agglomeration
* ADA U Aggregate dissemination area unique identifier


In [72]:
s5 = s2.GetName()
s5

'lda_000b16g_e'

In [139]:
s2[1016].GetField("DAUID")

11020208

In [84]:
s6 = s2[1010]
s6.GetFieldCount()

for i in range(s6.GetFieldCount()):
    print(s6.GetFieldDefn(i).GetName())

AttributeError: GetFieldDefn

In [69]:
for feature in s2:
    print(feature.GetField("CSDNAME"))

AttributeError: 'str' object has no attribute 'GetField'

In [53]:
for feature in s2:
    geom = feature.GetGeometryRef()
    print geom.Centroid().ExportToWkt()

SyntaxError: invalid syntax (<ipython-input-53-2897f5d30313>, line 3)